In [ ]:
!pip install sentence-transformers
!pip install tensorflow==2.3.0
!pip install transformers
!pip install torch

     |████████████████████████████████| 71kB 4.7MB/s 
     |████████████████████████████████| 1.1MB 12.4MB/s 
     |████████████████████████████████| 890kB 35.2MB/s 
     |████████████████████████████████| 3.0MB 46.7MB/s 
     |████████████████████████████████| 1.1MB 47.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=8b0d407099e666158d246e7862f69c092d0e97e1323ca85bfe1dd3704c36b433
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bc0ad0f81051d9387b6c8ddd03421a0aecb6ec8e7caead24265ffb04d86431cf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
from sentence_transformers import SentenceTransformer,util


model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

100%|██████████| 245M/245M [00:11<00:00, 22.1MB/s]


In [ ]:
!pip install python-docx
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 5.5MB 7.5MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=3dd78a42ded06eb29955adeffd2cc56488919ce66af91e786928be700cf4d477
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx
Mounted at /content/drive


In [ ]:
import docx

from docx import Document

document = Document("/content/drive/My Drive/leavepolicy.docx")  # Creating word reader object.
sentences = [abs]
for para in document.paragraphs:
    sentences.append(para.text)

sentences = sentences[1:]

In [ ]:
#sentences = text.split('\n')
sentences = [s.rstrip() for s in sentences]
sentences = [i for i in sentences if i] 
sentences = [s.lower() for s in sentences]
sentences

['sick leave is time off given by the company to allow employees to recover from an illness and take care of their health. sick leaves are crucial to allow employees to get the rest they need without worrying about losing pay. sick leave is a mandatory requirement in many countries to ensure the wellbeing of the employee.',
 'companies must provide 15 days of sick leave in a year to their employees. however, you must also be flexible with your sick leaves and allow employees to take longer ones if they have severe health issues. in many companies, any sick leaves that have not been taken by the end of the year can be carried forward to the next year. however, it’s important you ask your employees to take the day off if they are sick.',
 'casual leave is taken by an employee for travel, vacation, rest, and family events. such leaves are given to allow the employee to take time off for any life events they have like traveling to another country or weddings they have to attend. giving the

In [ ]:
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

In [ ]:
import scipy
import torch

queries =[input("Enter your value: ")]

top_k = 3
for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, sentence_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use torch.topk to find the highest 3 scores
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 3 most similar sentences in corpus:")
      

output = ''
for score, idx in zip(top_results[0], top_results[1]):
  print(sentences[idx], "(Score: %.4f)" % (score))
  if score >0.35:
        output += sentences[idx].strip()+' '

print('\n\n') 
output

Enter your value: what is maternity leave




Query: what is maternity leave

Top 3 most similar sentences in corpus:
maternity leave is provided to the new mother for a period of 7 to 17 weeks, depending on the country that the company is based out of. ideally, 14 weeks is a good amount of time to be given to the mother, allowing them to take care of their newborn for the first 3 months. (Score: 0.4659)
include these leaves within your leave policy by looking into the holidays that your government has mandated for your country. (Score: 0.4528)
you should also be open to providing extra leave days in case of any postnatal complications. (Score: 0.3890)





'maternity leave is provided to the new mother for a period of 7 to 17 weeks, depending on the country that the company is based out of. ideally, 14 weeks is a good amount of time to be given to the mother, allowing them to take care of their newborn for the first 3 months. include these leaves within your leave policy by looking into the holidays that your government has mandated for your country. you should also be open to providing extra leave days in case of any postnatal complications. '

In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
#import transformers

model1 = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

text =output
preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("extractive summary: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model1.generate(tokenized_text,
                                    num_beams=3,
                                    no_repeat_ngram_size=2,
                                    min_length=40,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nAbstractive Summarized text: \n",output)

extractive summary: 
 maternity leave is provided to the new mother for a period of 7 to 17 weeks, depending on the country that the company is based out of. ideally, 14 weeks is a good amount of time to be given to the mother, allowing them to take care of their newborn for the first 3 months. include these leaves within your leave policy by looking into the holidays that your government has mandated for your country. you should also be open to providing extra leave days in case of any postnatal complications.


Abstractive Summarized text: 
 maternity leave is provided to the new mother for a period of 7 to 17 weeks. ideally, 14 weeks is the best amount of time to be given, allowing the mother to take care of their newborn for the first 3 months. include these leaves in your leave policy by looking into the holidays that your government has mandated for your country.
